# Denoising Score Matching (DSM)

In [ ]:
from typing import Sequence
import functools
import numpy as np
import matplotlib.pyplot as plt
import jax
import jax.numpy as jnp
import flax.linen as nn
import haiku as hk
import optax
import chex
from tqdm import tqdm
from utils import BatchManager, load_dataset, save_samples

## Configuration and Data Loading

In [ ]:
# Define the configuration for each dataset
dataset_name = 'checkerboard'
dataset_configs = {
    'checkerboard': {
        'epochs': 5000
    },
    'gaussian_mixture': {
        'epochs': 5000
    },
    'pinwheel': {
        'epochs': 5000
    },
    'spiral': {
        'epochs': 5000
    }
}

config = dataset_configs[dataset_name]
config['batch_size'] = 128
config['learning_rate'] = 1e-3
config['std'] = 0.1
config['k'] = 100
config['mlp_layer_dim'] = [64, 64, 2]
X_train, X_test = load_dataset(dataset_name)

## Model Definition

In [ ]:
class MLP(nn.Module):
    layer_dim: Sequence[int]
    
    @nn.compact
    def __call__(self, x: jax.Array) -> jax.Array:
        for f in self.layer_dim[:-1]:
            x = nn.Dense(f)(x)
            x = nn.swish(x)
        x = nn.Dense(self.layer_dim[-1])(x)
        return x

model = MLP(layer_dim=config['mlp_layer_dim'])

## Training Preparation
Set up the optimizer, loss functions, and other training utilities.

In [ ]:
optimizer = optax.adam(learning_rate=config['learning_rate'])
prng_seq = hk.PRNGSequence(jax.random.PRNGKey(0))

def dsm_loss(params: chex.ArrayTree, batch: jax.Array, key: chex.PRNGKey, std: float, k: int) -> float:
    n, m = batch.shape
    batch = jnp.tile(batch[None, :, :], (k, 1, 1))
    noise = jax.random.normal(key, batch.shape) * std
    noised_batch = noise + batch
    fs = model.apply(params, noised_batch.reshape(k * n, m)).reshape(k, n, m)
    loss = jnp.sum(jnp.square(noised_batch + std**2 * fs - batch)) / k
    return loss

@jax.jit
def do_batch_update(batch: jax.Array, params: chex.ArrayTree, opt_state: optax.OptState, key: chex.PRNGKey) -> tuple[float, chex.ArrayTree, optax.OptState]:
    loss, grad = jax.value_and_grad(dsm_loss)(params, batch, key, std=0.1, k=100)
    updates, opt_state = optimizer.update(grad, opt_state)
    params = optax.apply_updates(params, updates)
    return loss, params, opt_state

## Training Loop

In [ ]:
params = model.init(next(prng_seq), X_train[:1, ...])
opt_state = optimizer.init(params)
bm = BatchManager(X_train, config['batch_size'], key=next(prng_seq))

In [ ]:
train_losses = []
test_losses = []
for epoch in tqdm(range(config['epochs']), "Epoch"):
    batch_loss = 0
    for _ in range(bm.num_batches):
        batch = next(bm)
        train_loss, params, opt_state = do_batch_update(batch, params, opt_state, key=next(prng_seq))
        batch_loss += train_loss
    test_loss = dsm_loss(params, X_test, key=next(prng_seq), std=config['std'], k=config['k'])
    train_losses.append(batch_loss / X_train.shape[0])
    test_losses.append(test_loss / X_test.shape[0])

## Training Results Visualization

In [ ]:
# Apply moving average filer to losses
window_size = 100
window = np.ones(window_size) / window_size
train_losses_f = np.convolve(train_losses, window, mode='valid')
test_losses_f = np.convolve(test_losses, window, mode='valid')
x_pos = np.arange(window_size // 2, window_size // 2 + train_losses_f.shape[0])

plt.plot(train_losses, label='Train Losses')
plt.plot(test_losses, label='Test Losses')
plt.plot(x_pos, train_losses_f, label='Smoothed Train Losses')
plt.plot(x_pos, test_losses_f, label='Smoothed Test Losses')
plt.xlabel('Epoch')
plt.ylabel('Denoising Score Matching Loss')
plt.legend()
plt.show()

## Sample Generation and Visualization

In [ ]:
@functools.partial(jax.jit, static_argnames=("num_steps",))
def langevin_sampling(params: chex.ArrayTree, key: chex.PRNGKey, step_size: float, initial_samples: jax.Array, num_steps: int) -> jax.Array:
    def scan_fn(carry, _):
        states, key = carry
        key, sk = jax.random.split(key)
        noise = jax.random.normal(sk, shape=states.shape)
        next_states = states + step_size * model.apply(params, states) + jnp.sqrt(2 * step_size) * noise
        return (next_states, key), None

    states = initial_samples
    (states, _), _ = jax.lax.scan(scan_fn, (states, key), jnp.arange(num_steps))
    return states

In [ ]:
samples = langevin_sampling(params, next(prng_seq), 5e-3, 2 * jax.random.normal(next(prng_seq), shape=(2000, 2)), 1000)

plt.scatter(samples[:, 0], samples[:, 1], marker='.', label='Sampled')
plt.scatter(X_train[:, 0], X_train[:, 1], alpha=0.2, marker='o', label='Train')
plt.scatter(X_test[:, 0], X_test[:, 1], alpha=0.2, marker='o', label='Test')
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.axis('equal')
plt.xlim([-4, 4])
plt.ylim([-4, 4])
plt.legend()
plt.show()

In [ ]:
# Save model outputs
save_samples('dsm', dataset_name, samples)